# Faire des classifieurs avec `scikit-learn`

Pour ce cours, aucune compétence en programmation n'est nécessaire, et tout peut se faire avec weka et le script de vectorisation fourni. Cependant, en dehors de ce cours, il est souvent utile de savoir utiliser un langage de script comme Python pour faire de la classification de documents par apprentissage artificiel.

Celà permet par exemple de modifier et d'étendre facilement les prétraitements, et de réaliser ces derniers et l'apprentissage et l'évaluation de classifieurs dans le même environnement. Cela facilite aussi le passage à l'échelle (par exemple pour des calculs lourds à moindre frais vous pouvez vous tourner ves [Google Collab](https://realpython.com/python-virtual-environments-a-primer/), mais attention à la politique d'usage des données) et vous trouverez plus facilement de la documentation et des cours.

## Considérations pratiques

Pour rendre ce document plus agréable à lire, il est présenté sous forme de notebook [Jupyter](https://jupyter.org/), que vous avez probablement déjà rencontré. C'est un format qui a ses avantages (pratique et lisible) et ses inconvénients (difficile à automatiser, mal versionnable, parfois sujet à erreurs). Tout ce qui est fait ici est tout à fait faisable dans un script indépendant et je vous encourage à écrire un tel script plutôt qu'à réutiliser des notebooks.

Pour pouvoir lancer vous-mêmes et modifier ce notebook, il vous faudra installer quelques dépendances. Je recommande de le faire dans un [environnement virtuel](https://realpython.com/python-virtual-environments-a-primer/) afin de ne pas polluer votre installation principale de Python, mais à défaut, vous pouvez lancer l'appel suivant à `pip` avec le flag `--user`.

**NE JAMAIS UTILISER PIP AVEC `sudo`**

Pour installer les dépendances, ouvrez un terminal et entrez

```console
pip install jupyter scikit-learn
```

Puis démarrez jupyter

```console
jupyter notebook demos/classier_with_sklearn.ipynb
```

Celà devrait vous ouvrir une fenêtre de navigateur sur le notebook, qui est alors prêt à être utilisé.

## Récupérer les données

On va travailler avec le petit corpus habituel. Récupérez le sur [la page du cours](https://loicgrobol.github.io/intro-fouille-textes) ou utilisez la commande suivante pour le télécharge dans un dossier `local/corpus` dans le même dossier que le notebook.

In [1]:
!mkdir -p local && wget -qO- "https://github.com/LoicGrobol/intro-fouille-textes/releases/download/stable/sample-data.tar.gz" | tar xz -C local --strip-components=2 data/expl3/corpus

Listons les fichiers du corpus

In [2]:
!ls -Rh local/corpus

local/corpus:
culture  société

local/corpus/culture:
doc0.txt   doc12.txt  doc15.txt  doc3.txt  doc6.txt  doc9.txt
doc10.txt  doc13.txt  doc1.txt   doc4.txt  doc7.txt
doc11.txt  doc14.txt  doc2.txt   doc5.txt  doc8.txt

local/corpus/société:
doc0.txt   doc12.txt  doc15.txt  doc3.txt  doc6.txt  doc9.txt
doc10.txt  doc13.txt  doc1.txt   doc4.txt  doc7.txt
doc11.txt  doc14.txt  doc2.txt   doc5.txt  doc8.txt


## Import

Commençons par importer le package [`scikit-learn`](https://scikit-learn.org), un très bon outil pour l'apprentissage artificiel en Python, sur lequel va s'appuyer la suite de cette démo.

Notez que le nom à utiliser pour l'import est bien `sklearn` pas `scikit-learn`. On importe également les sous-modules qui nous seront utiles.

In [3]:
import sklearn
import sklearn.datasets

On import maintenant les données : `scikit-learn` a pour ça une commande toute prête, [`sklearn.datasets.load_files`](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_files.html), qui prend en entrée exactement le même format que le script de vectorisation — un dossier par classe, un fichier texte par document. `shuffle` sert ici à mélanger les documents pour ne pas avoir tous les documents d'une même classe les uns à la suite des autres.

In [4]:
dataset = sklearn.datasets.load_files("local/corpus", encoding="utf-8", shuffle=True)

L'objet renvoyé est de type [`sklearn.utils.Bunch`](https://scikit-learn.org/stable/modules/generated/sklearn.utils.Bunch.html) et est prévu pour permettre facilement d'accéder aux documents via son attribut `data`. Ici les 100 premiers caractères du premier document :

In [5]:
dataset.data[0][:100]

"La française recouvre d'abord l'ensemble des pratiques qui se trouvent sur le territoire français. M"

Les classes sont données par `target` :

In [6]:
dataset.target[:5]

array([0, 1, 0, 0, 1])

Les indices correspondants aux noms dans `target_names`

In [7]:
dataset.target_names

['culture', 'société']

## Vectorisation

L'étape suivant est de vectoriser notre corpus : transformer chaque document en une liste de nombres comme on l'a vu dans le cours. `scikit-learn` fournit l'utilitaire [`sklearn.feature_extraction.text.CountVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html), très complet qui permet d'appliquer une segmentation basique et de transformer des chaînes de caractères en sacs de mots.

Commençons par importer le sous-module dans lequel il se trouve

In [8]:
import sklearn.feature_extraction.text

Puis appliquons-le à nos données

In [22]:
vectorizer = sklearn.feature_extraction.text.CountVectorizer()
counts = vectorizer.fit_transform(dataset.data)
print(f"Le corpus a {counts.shape[0]} lignes et {counts.shape[1]} colonnes")
print("Et voici un échantillon :")
counts[:6, :15].todense()

Le corpus a 32 lignes et 11415 colonnes
Et voici un échantillon :


matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

Le résultat est une matrice, stockée sous forme creuse pour économiser de la mémoire (on ne stocke que les valeurs non-nulles), d'où l'appel à `todense` pour l'afficher.

`CountVectorizer` a de nombreuses options de prétraitement, et je vous encourage à aller lire [sa documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) et à expérimenter avec les paramètres. Par exemple pour filtrer les hapax, on peut fixer `min_df` *minimum document frequency* à 2 :

In [10]:
vectorizer = sklearn.feature_extraction.text.CountVectorizer(min_df=2)
counts = vectorizer.fit_transform(dataset.data)
 

Notons enfin qu'on utilise ici la fonction `fit_transformer` qui combine deux opérations

- `fit`, qui construit le vocabulaire
- `transform`, qui vectorise les données en utilisant ce vocabulaire

On peut aussi les utiliser à part. Par exemple ici on ne construit le vocabulaire que sur les 10 premiers textes, mais on vectorise tout :

In [11]:
vectorizer = sklearn.feature_extraction.text.CountVectorizer(min_df=2)
vectorizer.fit(dataset.data[:10])
counts_only10 = vectorizer.transform(dataset.data)
print(f"Le corpus sans les hapax et avec vocabulaire réduit a {counts_only10.shape[0]} lignes et {counts_only10.shape[1]} colonnes") 

Le corpus sans les hapax et avec vocabulaire réduit a 32 lignes et 1370 colonnes


Quelle que soit la façon dont on le fait, un fois que le vectoriser a été `fit`é, on peut le réappliquer à d'autres données : en particulier pour votre corpus de test.

Pour construire un corpus de test on va se contenter d'utiliser un simple split aléatoire, mais il y a aussi moyen de faire de la validation croisée.

In [12]:
import sklearn.model_selection

In [20]:
train_set, test_set, y_train, y_test = sklearn.model_selection.train_test_split(dataset.data, dataset.target, test_size=0.33)
vectorizer = sklearn.feature_extraction.text.CountVectorizer(min_df=2)
train_set_counts = vectorizer.fit_transform(train_set)
test_set_counts = vectorizer.transform(test_set)
test_set_counts.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        ...,
        [1, 1, 0, ..., 0, 1, 1],
        [4, 3, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

## Entraîner des classifieurs